In [21]:
import os
import pandas as pd

directory = os.getcwd() + "/output/rk4functions/"
dfs = []

executions = 30

for function_ in ['lotka-volterra']:
    for type_ in ['0', '1', '2', '3', '4']:
        df = pd.read_csv(directory + type_ + "/" + function_ + "/res.csv", sep = ";")
        df['function'] = [function_] * len(df)
        df['approach'] = [type_] * len(df)
#         print(df)
        dfs.append(df)
dfs

[    seed       training           test  validation  \
 0      7   6.497260e+01   1.409160e+01    0.095619   
 1     13   2.932690e+02   7.314410e+00    0.221646   
 2     19   2.932690e+02   7.314410e+00    0.221646   
 3      1   1.608320e+02   7.444010e+00    0.065612   
 4     25   7.617540e+00   5.873970e+00    0.190308   
 5     14   1.532740e+03   4.746930e+00    0.048022   
 6     20   9.709260e+04   1.586510e+01    0.116232   
 7      2   1.954770e+02   5.734250e+00    0.082027   
 8      8            inf   7.822300e+00    0.363304   
 9     26   1.040280e+02   3.851430e+00    0.141476   
 10    15            inf   6.704900e+00    0.351040   
 11    21   4.150380e+02   4.168220e+00    0.277117   
 12     3   2.830690e+02   6.372630e+00    0.097153   
 13     9   3.870910e+01   3.809280e+00    0.454411   
 14    27   3.275550e+02   1.065940e+01    0.227980   
 15    16   1.646390e+02   8.484190e+00    0.124354   
 16    22   1.646390e+02   8.484190e+00    0.124354   
 17     4 

In [22]:
results = pd.concat(dfs)
# results = results[results.seed.isin([1, 2, 3])]
results.columns = ['seed', 'training', 'validation', 'test', 'model', 'constants', 'function', 'approach']
results.describe()

,seed,training,validation,test,constants
count,142.000000,142.00000,1.420000e+02,142.000000,0.0
mean,15.507042,inf,3.258930e+127,0.150436,NaN
std,8.716778,NaN,3.883463e+128,0.124909,NaN
min,1.000000,4.06682,4.594710e-02,0.015137,NaN
25%,8.000000,164.63900,6.033635e+00,0.056590,NaN
50%,15.500000,243.62700,7.507180e+00,0.098053,NaN
75%,23.000000,497.28500,9.084435e+00,0.208379,NaN
max,30.000000,inf,4.627680e+129,0.580109,NaN


In [23]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from __future__ import division
import random
from sympy import *

IF_ELSE = set(['if-else'])
BINARY_OPERATORS = set(['+', '-', '*', '/', '^', '<', '<=', '==', '>=', '>', '!=', 'and', 'or'])
UNARY_OPERATORS = set(['log', 'sqrt', 'exp', 'sin'])

### POSTFIX ===> INFIX ###
'''
1) When see an operand, push
2) When see an operator, pop out two numbers, connect them into a substring and push back to the stack
3) the top of the stack is the final infix expression.
'''
def postfix_to_infix(formula):
    stack = []
    for ch in formula.split():
        if ch in BINARY_OPERATORS:
            if ch == "^":
                ch = "**"
            b = stack.pop()
            a = stack.pop()
            expr = "(" + a + ch + b + ")"
            stack.append(expr)
        elif ch in UNARY_OPERATORS:
            a = stack.pop()
            expr = ch + "(" + a + ")"
            stack.append(expr)
        elif ch in IF_ELSE:
            c = stack.pop()
            b = stack.pop()
            a = stack.pop()
            expr = "if(" + c + "){\n" + a + "\n}else{\n" + b + "\n}"
            stack.append(expr)
        else:
            stack.append(ch)

    return stack[-1]

def postfix_to_infix_pyton(formula, constants):
    stack = []
    c_counter = 0
    for ch in formula.split():
        if ch in BINARY_OPERATORS:
            if ch == "^":
                ch = "**"
            b = stack.pop()
            a = stack.pop()
            expr = "(" + a + ch + b + ")"
            stack.append(expr)
        elif ch in UNARY_OPERATORS:
            a = stack.pop()
            expr = ch + "(" + a + ")"
            stack.append(expr)
        elif ch in IF_ELSE:
            c = stack.pop()
            b = stack.pop()
            a = stack.pop()
            expr = "if(" + c + "):\n" + a + "\nelse:\n" + b + "\n"
            stack.append(expr)
        elif ch in ["_c_"]:            
            stack.append(str(constants[c_counter]))
            c_counter = c_counter + 1
        else:
            stack.append(ch)

    return stack[-1]

# if __name__ == '__main__':
#     infix = postfix_to_infix_pyton("cost income - cost return / + cost income - return income / + + cost return cost + - cost return * cost / + * income income return / ^ return return return * income cost * * * * /")
#     print("PG output: " + infix)
#     # print("Simplify: " + str(simplify(infix)))


In [24]:
infix = []

for i, model in results.iterrows():
    infix.append(postfix_to_infix_pyton(model.model, []))

In [25]:
results['infix'] = infix

In [26]:
# results.to_csv("PG/output/activity-000/results.csv", index = None)

In [27]:
print(list(results[results.test.min() == results.test].model))

[' e(0) = x z 2 + -1 * / 0 y z z y / - * / y x * -4 + + * |  e(1) = y y y * - y y * -4 y + / + |  e(2) = 1 z * 1 z x / 1 * y -4 + * ^ / 1 x - -5 y * -3 - / / | ']


In [28]:
results.describe()

,seed,training,validation,test,constants
count,142.000000,142.00000,1.420000e+02,142.000000,0.0
mean,15.507042,inf,3.258930e+127,0.150436,NaN
std,8.716778,NaN,3.883463e+128,0.124909,NaN
min,1.000000,4.06682,4.594710e-02,0.015137,NaN
25%,8.000000,164.63900,6.033635e+00,0.056590,NaN
50%,15.500000,243.62700,7.507180e+00,0.098053,NaN
75%,23.000000,497.28500,9.084435e+00,0.208379,NaN
max,30.000000,inf,4.627680e+129,0.580109,NaN


In [29]:
import numpy as np

for function_ in results.function.unique():
    print(function_)
    for approach_ in results.approach.unique():
        f_df = results[(results['function'] == function_) & (results['approach'] == approach_)]
        f_df = f_df['test']
        print(#function_, 
              " & ",
              approach_, 
              " & ",
              "{:.5e}".format(np.min(f_df)), " & ",
              "{:.5e}".format(np.median(f_df)), " & ",
              "{:.5e}".format(np.mean(f_df)), " & ",
              "{:.5e}".format(np.std(f_df)), " & ",
              "{:.5e}".format(np.max(f_df)), " \\\\")

lotka-volterra
 &  0  &  1.51365e-02  &  1.35489e-01  &  1.86549e-01  &  1.27179e-01  &  5.40145e-01  \\
 &  1  &  2.46875e-02  &  7.48830e-02  &  1.09877e-01  &  8.08220e-02  &  2.98770e-01  \\
 &  2  &  3.55136e-02  &  1.16569e-01  &  1.69676e-01  &  1.45395e-01  &  5.80109e-01  \\
 &  3  &  3.54433e-02  &  8.51010e-02  &  1.67386e-01  &  1.46217e-01  &  5.41707e-01  \\
 &  4  &  2.00224e-02  &  8.52490e-02  &  1.07150e-01  &  6.52764e-02  &  2.18263e-01  \\


In [30]:
import numpy as np

for approach_ in results.approach.unique():
    print(approach_)
    for function_ in results.function.unique():
        f_df = results[(results['function'] == function_) & (results['approach'] == approach_)]
        f_df = f_df['test']
        print(function_, "{:.5e}".format(np.mean(f_df)))

0
lotka-volterra 1.86549e-01
1
lotka-volterra 1.09877e-01
2
lotka-volterra 1.69676e-01
3
lotka-volterra 1.67386e-01
4
lotka-volterra 1.07150e-01
